In [1]:
import pandas as pd
import numpy as np
import re
from scrapers.GeneralScraper import Scraper
from NFL_RefMaps import TableColumns,TeamDictionary

In [2]:
link = 'https://www.fantasypros.com/nfl/projections/dst.php?week=1&scoring=HALF'
page_soup = Scraper(link).page_soup

In [3]:
data = page_soup.find("tbody")

In [4]:
table = 'dst_projections'
metrics = [TableColumns().fantasy_pros[table]]
players = data.findAll("tr",{"class":re.compile(r'mpb-player-')})

In [5]:
for player in players:
    stats = player.findAll("td")
    name = stats[0].a.text
#     team = stats[0].text.strip().split()[-1]
    team = TeamDictionary().nfl_api[name]
    a = stats[0].findAll("a")
    pid = a[1].get('class')[1]
    mets = np.array([pid,name,team])
    for stat in stats[1:]:
        mets = np.append(mets,stat.text)
    print(len(mets))
    print(mets)
    metrics.append(mets)
df = np.vstack(metrics)
pd.DataFrame(data=df[1:,1:],index=df[1:,0],columns=df[0,1:])

12
['fp-id-8020' 'Baltimore Ravens' 'BAL' '2.6' '1.2' '0.6' '0.9' '0.2' '0.0'
 '17.2' '290.2' '8.4']
12
['fp-id-8210' 'New York Jets' 'NYJ' '2.8' '1.1' '0.6' '0.9' '0.2' '0.0'
 '19.0' '320.5' '8.2']
12
['fp-id-8070' 'Cleveland Browns' 'CLE' '2.7' '0.9' '0.7' '1.4' '0.2' '0.0'
 '19.3' '309.2' '8.0']
12
['fp-id-8080' 'Dallas Cowboys' 'DAL' '2.9' '0.9' '0.7' '1.2' '0.2' '0.0'
 '17.8' '317.9' '7.9']
12
['fp-id-8260' 'Seattle Seahawks' 'SEA' '2.7' '1.0' '0.6' '1.0' '0.2' '0.0'
 '17.7' '316.1' '7.9']
12
['fp-id-8190' 'New Orleans Saints' 'NO' '3.5' '0.9' '0.7' '1.1' '0.2'
 '0.0' '21.2' '350.8' '7.8']
12
['fp-id-8230' 'Philadelphia Eagles' 'PHI' '2.5' '1.0' '0.6' '0.9' '0.2'
 '0.0' '17.2' '312.3' '7.7']
12
['fp-id-8100' 'Detroit Lions' 'DET' '3.2' '0.9' '0.7' '1.0' '0.2' '0.0'
 '20.6' '322.6' '7.5']
12
['fp-id-8270' 'San Francisco 49ers' 'SF' '2.9' '1.1' '0.7' '1.0' '0.2'
 '0.0' '24.6' '366.1' '7.5']
12
['fp-id-8250' 'Los Angeles Chargers' 'LAC' '2.6' '0.9' '0.6' '0.8' '0.1'
 '0.0' '17.7' '32

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 13 and the array at index 1 has size 12

In [17]:
import pandas as pd
import numpy as np
import re
from scrapers.GeneralScraper import Scraper
from NFL_RefMaps import TableColumns,TeamDictionary

class LoadProjections():

    def __init__(self,week):

        self.week = week
        self.projections = self.get_all_projections()

    def get_projections(self,pos):
        link = "https://www.fantasypros.com/nfl/projections/"+pos+".php?week="+str(self.week)+"&scoring=HALF"
        print(link)
        page_soup = Scraper(link).get_soup()
        data = page_soup.find("tbody")

        table = pos+'_projections'
        metrics = [TableColumns().fantasy_pros[table]]
        players = data.findAll("tr",{"class":re.compile(r'mpb-player-')})
        for player in players:
            stats = player.findAll("td")
            name = stats[0].a.text
            print(name)
            team = 'n/a'
            if pos == 'dst':
                team = TeamDictionary().nfl_api[name]
            else:
                print(stats[0].text)
                ts = stats[0].text.strip().split()
                if len(ts) == 0:
                    print("Error detected, skipping this row")
                    break
                print(ts)
                print(len(ts))
                team = stats[0].text.strip().split()[-1]
            a = stats[0].findAll("a")
            pid = a[1].get('class')[1]
            mets = np.array([pid,name,team,self.week])
            for stat in stats[1:]:
                mets = np.append(mets,stat.text)
            metrics.append(mets)
        df = np.vstack(metrics)

        return pd.DataFrame(data=df[1:,1:],index=df[1:,0],columns=df[0,1:])

    def get_all_projections(self):
        projections = {
            'qb':'',
            'rb':'',
            'wr':'',
            'te':'',
            'dst':'',
            'k':''
        }
        for key,val in projections.items():
            projections[key] = self.get_projections(key)

        return projections


In [18]:
proj = LoadProjections(week=1)

https://www.fantasypros.com/nfl/projections/qb.php?week=1&scoring=HALF
Patrick Mahomes
Patrick Mahomes KC 
['Patrick', 'Mahomes', 'KC']
3
Deshaun Watson
Deshaun Watson HOU 
['Deshaun', 'Watson', 'HOU']
3
Drew Brees
Drew Brees NO 
['Drew', 'Brees', 'NO']
3
Lamar Jackson
Lamar Jackson BAL 
['Lamar', 'Jackson', 'BAL']
3
Jameis Winston
Jameis Winston TB 
['Jameis', 'Winston', 'TB']
3
Dak Prescott
Dak Prescott DAL 
['Dak', 'Prescott', 'DAL']
3
Russell Wilson
Russell Wilson SEA 
['Russell', 'Wilson', 'SEA']
3
Cam Newton
Cam Newton CAR 
['Cam', 'Newton', 'CAR']
3
Carson Wentz
Carson Wentz PHI 
['Carson', 'Wentz', 'PHI']
3
Aaron Rodgers
Aaron Rodgers GB 
['Aaron', 'Rodgers', 'GB']
3
Baker Mayfield
Baker Mayfield CLE 
['Baker', 'Mayfield', 'CLE']
3
Jared Goff
Jared Goff LAR 
['Jared', 'Goff', 'LAR']
3
Mitch Trubisky
Mitch Trubisky CHI 
['Mitch', 'Trubisky', 'CHI']
3
Tom Brady
Tom Brady NE 
['Tom', 'Brady', 'NE']
3
Ben Roethlisberger
Ben Roethlisberger PIT 
['Ben', 'Roethlisberger', 'PIT']
3
Kyl

DeAndre Hopkins
DeAndre Hopkins HOU 
['DeAndre', 'Hopkins', 'HOU']
3
Davante Adams
Davante Adams GB 
['Davante', 'Adams', 'GB']
3
Michael Thomas
Michael Thomas NO 
['Michael', 'Thomas', 'NO']
3
Odell Beckham Jr.
Odell Beckham Jr. CLE 
['Odell', 'Beckham', 'Jr.', 'CLE']
4
Julio Jones
Julio Jones ATL 
['Julio', 'Jones', 'ATL']
3
JuJu Smith-Schuster
JuJu Smith-Schuster PIT 
['JuJu', 'Smith-Schuster', 'PIT']
3
Mike Evans
Mike Evans TB 
['Mike', 'Evans', 'TB']
3
Tyler Lockett
Tyler Lockett SEA 
['Tyler', 'Lockett', 'SEA']
3
Adam Thielen
Adam Thielen MIN 
['Adam', 'Thielen', 'MIN']
3
Keenan Allen
Keenan Allen LAC 
['Keenan', 'Allen', 'LAC']
3
Amari Cooper
Amari Cooper DAL 
['Amari', 'Cooper', 'DAL']
3
Julian Edelman
Julian Edelman NE 
['Julian', 'Edelman', 'NE']
3
Tyler Boyd
Tyler Boyd CIN 
['Tyler', 'Boyd', 'CIN']
3
Brandin Cooks
Brandin Cooks LAR 
['Brandin', 'Cooks', 'LAR']
3
Stefon Diggs
Stefon Diggs MIN 
['Stefon', 'Diggs', 'MIN']
3
Kenny Golladay
Kenny Golladay DET 
['Kenny', 'Golladay

Travis Kelce
Travis Kelce KC 
['Travis', 'Kelce', 'KC']
3
George Kittle
George Kittle SF 
['George', 'Kittle', 'SF']
3
Zach Ertz
Zach Ertz PHI 
['Zach', 'Ertz', 'PHI']
3
Evan Engram
Evan Engram NYG 
['Evan', 'Engram', 'NYG']
3
O.J. Howard
O.J. Howard TB 
['O.J.', 'Howard', 'TB']
3
Vance McDonald
Vance McDonald PIT 
['Vance', 'McDonald', 'PIT']
3
Jared Cook
Jared Cook NO 
['Jared', 'Cook', 'NO']
3
David Njoku
David Njoku CLE 
['David', 'Njoku', 'CLE']
3
Hunter Henry
Hunter Henry LAC 
['Hunter', 'Henry', 'LAC']
3
Mark Andrews
Mark Andrews BAL 
['Mark', 'Andrews', 'BAL']
3
Darren Waller
Darren Waller OAK 
['Darren', 'Waller', 'OAK']
3
Delanie Walker
Delanie Walker TEN 
['Delanie', 'Walker', 'TEN']
3
Eric Ebron
Eric Ebron IND 
['Eric', 'Ebron', 'IND']
3
Will Dissly
Will Dissly SEA 
['Will', 'Dissly', 'SEA']
3
Austin Hooper
Austin Hooper ATL 
['Austin', 'Hooper', 'ATL']
3
Jimmy Graham
Jimmy Graham GB 
['Jimmy', 'Graham', 'GB']
3
Vernon Davis
Vernon Davis WAS 
['Vernon', 'Davis', 'WAS']
3
Ky

In [25]:
proj.projections['k']

,name,team,week,fg,fga,xpt,fantasy_points
fp-id-16712,Harrison Butker,KC,5,1.9,2.1,3.3,8.9
fp-id-16026,Wil Lutz,NO,5,2.2,2.4,2.3,8.8
fp-id-9534,Robbie Gould,SF,5,2.1,2.3,2.2,8.5
fp-id-15756,Ka'imi Fairbairn,HOU,5,2.0,2.2,2.5,8.4
fp-id-16540,Jake Elliott,PHI,5,2.0,2.2,2.5,8.4
fp-id-14003,Jason Myers,SEA,5,1.9,2.2,2.5,8.1
fp-id-11345,Greg Zuerlein,LAR,5,2.0,2.2,2.2,8.1
fp-id-11465,Justin Tucker,BAL,5,2.0,2.2,2.1,8.0
fp-id-12378,Brett Maher,DAL,5,1.8,2.2,2.3,7.9
fp-id-19028,Joey Slye,CAR,5,1.9,2.2,2.0,7.6
